In [2]:
import re
import pandas as pd

beyonce_df = pd.read_csv('/Users/mertakcay/Projects/Knowledge Graphs/Neo4j/context.csv')
context_list = beyonce_df['context'].apply(lambda x : re.sub(r'[^\w\s]', '', x))

In [3]:
from gpt4all import GPT4All
model = GPT4All("/Users/mertakcay/Projects/Knowledge Graphs/KnowledgeGraphBuilder/mistral-7b-instruct-v0.1.Q4_0.gguf")

In [4]:
example_term = """
Beyoncé, Destiny's Child
"""
example_groups = """
Person: Beyoncé, Marc Nelson
Group: Destiny's Child, Boyz II Men
"""
example_relation ="""
part_of (Person, Group): Marc Nelson part_of Boyz II Men
"""

In [8]:
Step 2: Group term list like {example_groups} Step 3: Extract relationship between groups like {example_relation} from {context}

SyntaxError: invalid syntax (2573736816.py, line 1)

In [4]:
for context in context_list:
    prompt_ner = f'''
    Extract <Subject, Predicate, Object> from {context} in json format
    '''
    output = model.generate(prompt_ner, max_tokens=8192)
    print(output)
    print("----")


    {
        "Subject": "Beyoncé Giselle KnowlesCarter",
        "Predicate": "born September 4, 1981 is an American singer-songwriter record producer and actress born and raised in Houston, Texas. She performed in various singing and dancing competitions as a child and rose to fame in the late 1990s as lead singer of RB girlgroup Destinys Child.",
        "Object": null
    }
----

    Subject: Beyoncé
    Predicate: released her second solo album BDay, ventured into acting with a Golden Globenominated performance in Dreamgirls, starred in The Pink Panther and Obsessed, married rapper Jay Z, portrayed Etta James in Cadillac Records, took over management of her career, released fourth album 4, released critically acclaimed fifth studio album Beyoncé
    Object: BDay, Dreamgirls, The Pink Panther, Obsessed, Jay Z, Etta James, Cadillac Records, 4, Beyoncé
----

    {
        "Subject": "Beyoncé",
        "Predicate": [
            "creates songs that are often characterized by themes o

Exception ignored on calling ctypes callback function: <function LLModel._callback_decoder.<locals>._raw_callback at 0x137fc99d0>
Traceback (most recent call last):
  File "/Users/mertakcay/.pyenv/versions/3.9.18/envs/knowledge/lib/python3.9/site-packages/gpt4all/_pyllmodel.py", line 393, in _raw_callback
    def _raw_callback(token_id: int, response: bytes) -> bool:
KeyboardInterrupt: 


 {
        "Subject": "Dest
----
